In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.fft import rfft,rfftfreq

sns.set()

In [ ]:

SAMPLE_RATE = 16  # Hertz - Nyquist theorem : sampling rate at least 2 x highest signal freq. 
DURATION = 5  # Seconds

def generate_sine_wave(freq, sample_rate, duration):
    x = np.linspace(0, duration, sample_rate * duration, endpoint=False)
    frequencies = x * freq
    # 2pi because np.sin takes radians
    y = np.sin((2 * np.pi) * frequencies)
    return x, y


f1 = 2
f2 = 4

# Generate a sine wave  of f Hz that lasts for 5 seconds
x, y = generate_sine_wave(f1, SAMPLE_RATE, DURATION)
x2,y2 = generate_sine_wave(f2,SAMPLE_RATE,DURATION)

plt.figure(figsize=(18,12))
plt.plot(x, y)
plt.plot(x2,y2)



In [ ]:
signal = y + y2 * 0.5

N = SAMPLE_RATE * DURATION

yf = rfft(signal)
xf = rfftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf) / N)
plt.ylabel('power')
plt.xlabel('frequency')
plt.show()

In [ ]:
faked_yearly_data = np.array([0,1,2,3,4,4,3,2,1,0] * 10)
faked_yearly_data = faked_yearly_data / faked_yearly_data.max() #normalize

plt.plot(faked_yearly_data,'o--')
plt.xlabel('year')

In [ ]:

sample_rate = 10 # 10 samples per decennium

faked_yearly_data = faked_yearly_data - faked_yearly_data.mean() # remove DC-component

yf = rfft(faked_yearly_data)
xf = rfftfreq(len(faked_yearly_data),1 / sample_rate) #normalize so that we can have low sampling rate (Nyquist)

plt.plot(xf,np.abs(yf) / len(faked_yearly_data))
plt.xlabel('freq [ 1/ decade]')


In [ ]:
# next example

In [ ]:
def build_signal(f_arr,power_arr):
    sig = np.zeros_like(SAMPLE_RATE * DURATION)
    
    for i,f in enumerate(f_arr):
        
        xf,yf = generate_sine_wave(f,SAMPLE_RATE,DURATION)
        sig = sig + yf * power_arr[i]
        
    return xf,sig


In [ ]:
def normalize_sig (sig):
    return np.int16((sig / sig.max()) * 32767)


In [ ]:
SAMPLE_RATE = 44100
DURATION = 5
freq_boost_factor = 13000

f_arr = np.array([1]) * freq_boost_factor
power_arr = np.array([1])

print ('component frequencies:',f_arr)
xf,yf = build_signal(f_arr,power_arr)


In [ ]:
subset = 100

plt.figure(figsize=(18,12))
plt.plot(xf[:subset],yf[:subset],color='crimson',lw=5,label='resultant signal')

for i,f in enumerate(f_arr):
    x_tmp,y_tmp = generate_sine_wave(f,SAMPLE_RATE,DURATION)
    plt.plot(x_tmp[:subset],y_tmp[:subset] * power_arr[i],'--',label='{} Hz'.format(f))
    
plt.legend(loc='upper right')    
plt.xlabel('time [s]')

In [ ]:
audible_sig = normalize_sig(yf)
plt.plot(xf[:subset],audible_sig[:subset])

In [ ]:
from scipy.io.wavfile import write

write("mysinewave.wav", SAMPLE_RATE, audible_sig)

In [ ]:
def concat_freqs(f_arr,power_arr):
    sig = np.zeros(SAMPLE_RATE * DURATION * len(f_arr))
    idx = 0
    
    for i,f in enumerate(f_arr):
        xf,yf = generate_sine_wave(f,SAMPLE_RATE,DURATION)
        yf = yf * power_arr[i]
        
        sig[idx:idx + SAMPLE_RATE * DURATION] = yf
        idx += SAMPLE_RATE * DURATION
        
    return sig
        

In [ ]:
DURATION = 1


f_arr = np.array([20,40,60,80,100,10000,14000,15000,
                 16000,17000,18000])

power_arr = np.repeat(1,len(f_arr))

varying_freq_signal = concat_freqs(f_arr,power_arr)

In [ ]:
varying_freq_signal = normalize_sig(varying_freq_signal)

write('varying_freq_signal.wav',SAMPLE_RATE,varying_freq_signal)

In [ ]:
yf = rfft(varying_freq_signal)
yf = yf / yf.max()

yf = yf - yf.mean()
xf = rfftfreq(len(varying_freq_signal),1 / SAMPLE_RATE)

In [ ]:
plt.figure(figsize=(18,12))
plt.title('Varying audible signal - frequency domain')
plt.plot(xf,np.abs(yf))
plt.ylabel('power')
plt.xlabel('Hz')


In [ ]:
plt.plot(varying_freq_signal[:44100])

In [ ]:
# perfect fifth

f1 = 600
f2 = 400 # 2/3 of f1

_,sig1 = generate_sine_wave(f1,SAMPLE_RATE,DURATION)
_,sig2 = generate_sine_wave(f2,SAMPLE_RATE,DURATION)

perf_5 = sig1 + sig2

write('perfect_5th.wav',SAMPLE_RATE,perf_5)

In [ ]:
perf_5_yf = rfft(perf_5)
perf_5_xf = rfftfreq(len(perf_5), 1 / SAMPLE_RATE)

plt.plot(perf_5_xf[:1000],np.abs(perf_5_yf[:1000]))

In [ ]:
# piano frequencies
import re

SAMPLE_RATE = 44100

frequencies = []
with open('piano_freq.txt','r') as f:
    
    for line in f:
        result = re.match('[0-9]+\.[0-9]+',line.split()[-1])
        if result:
            frequencies.append(result.group(0))
            
frequencies = np.array(frequencies).astype(float)
power_arr = np.repeat(1,len(frequencies))

piano_keys = concat_freqs(frequencies,power_arr)

piano_keys = normalize_sig(piano_keys)

write('piano_keys.wav',SAMPLE_RATE,piano_keys)



In [ ]:
yf = rfft(piano_keys)
yf = yf - yf.mean()

xf = rfftfreq(len(piano_keys), 1 / SAMPLE_RATE)

plt.figure(figsize=(18,12))
plt.plot(xf,np.abs(yf) / np.abs(yf).max())